In [ ]:
"""
Analysis of Nematostella planula single cell RNAseq data using ScanPy
Marc Meynadier
"""

In [ ]:
!date

In [ ]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy.sparse
import scanpy.external as sce
import os

import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import (KNeighborsClassifier,NeighborhoodComponentsAnalysis)
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from matplotlib import rcParams
#import plotly.tools as tls

import seaborn as sns
sns.set(style="whitegrid")

import collections
import scipy.sparse as sp_sparse
import h5py

In [ ]:
sc.logging.print_header()
sc.settings.set_figure_params(dpi=150, facecolor='white')

In [ ]:
# Loading planula data

pla2dPath = "../../../../species/Nematostella/analysis/STARmapping/steger2022/GSE200198/GSM6021902_planula2d/"
pla3dPath = "../../../../species/Nematostella/analysis/STARmapping/steger2022/GSE200198/GSM6021903_planula3d/"
pla4dPath = "../../../../species/Nematostella/analysis/STARmapping/steger2022/GSE200198/GSM6021904_planula4d/"
pla4d2cPath = "../../../../species/Nematostella/analysis/STARmapping/steger2022/GSE200198/GSM6021905_planula4d2c/"
pla2dData = sc.read_10x_mtx(pla2dPath) ; pla2dData.var_names_make_unique()
pla3dData = sc.read_10x_mtx(pla3dPath) ; pla3dData.var_names_make_unique()
pla4dData = sc.read_10x_mtx(pla4dPath) ; pla4dData.var_names_make_unique()
pla4d2cData = sc.read_10x_mtx(pla4d2cPath) ; pla4d2cData.var_names_make_unique()

In [ ]:
# Metadata
pla2dData.obs['type']="planula"
pla2dData.obs['sample']="Planula 2d"

pla3dData.obs['type']="planula"
pla3dData.obs['sample']="Planula 3d"

pla4dData.obs['type']="planula"
pla4dData.obs['sample']="Planula 4d"

pla4d2cData.obs['type']="planula"
pla4d2cData.obs['sample']="Planula 4d cryo"

In [ ]:
# Merging
adata = pla2dData.concatenate(pla3dData,pla4dData,pla4d2cData,join='outer') 
print(adata.obs['sample'].value_counts())

In [ ]:
# Mitochondrial genes
mitoGenes = pd.read_csv('../../../../species/Nematostella/raw/NematostellaMitoGenes.txt',sep='\t')
mitoGenes = list(set(list(mitoGenes.iloc[:,0])))

adata.var['mt'] = False
adata.var = adata.var.reset_index()
for idx, row in adata.var.iterrows():
    for j in mitoGenes:
        if row['index'] == j:
            adata.var.at[idx,'mt'] = True
adata.var = adata.var.set_index('index')

print(adata.var)
adata.var.mt.value_counts()

In [ ]:
# Mitochondrial quality check
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=True, inplace=True)
mito_genes = adata.var['mt'].to_numpy()
# For each cell compute fraction of counts in mito genes vs. all genes
adata.obs['percent_mt2'] = np.sum(adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
# Add the total counts per cell as observations-annotation 
adata.obs['n_counts'] = adata.X.sum(axis=1).A1
adata

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts'],
             jitter=0.4, groupby = 'sample', rotation= 45, ncols=2)
sc.pl.violin(adata, ['pct_counts_mt', 'percent_mt2','log1p_total_counts_mt'],
             jitter=0.4, groupby = 'sample', rotation= 45, ncols=2)

In [ ]:
# Plot quality check measures
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt', color="sample")
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color="sample")
sc.pl.scatter(adata, x='log1p_total_counts', y='log1p_n_genes_by_counts', color="sample")

In [ ]:
# Filtering by genes counts and mitochrondial genes counts
adata = adata[adata.obs.n_genes_by_counts < 9000, :]
adata = adata[adata.obs.pct_counts_mt < 1, :]
adata

In [ ]:
# Filtering by number of counts and number of cells
sc.pp.filter_cells(adata, min_counts=500)
sc.pp.filter_genes(adata, min_cells=2)
print(adata.n_obs,adata.n_vars)

In [ ]:
# Plotting percentage of counts per gene 
sc.pl.highest_expr_genes(adata, n_top=50)

In [ ]:
# Exporting raw adata
rPath = os.getcwd()
outputPath = rPath+'/../../../../species/Nematostella/analysis/ScanPy/'
adata.write_h5ad(outputPath+'Nematostella_planula_raw.h5ad')

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

adata.raw = adata

adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
# Correcting batch effet
sce.pp.harmony_integrate(adata,'sample',max_iter_harmony=20)

In [ ]:
adata

In [ ]:
#plot the amount of variance explained by each PC
sc.pl.pca_variance_ratio(adata, log=True, n_pcs = 50)

In [ ]:
# Computing neighbors graph
sc.pp.neighbors(adata, n_neighbors=50, n_pcs=26, knn=True, use_rep='X_pca_harmony')

# Clustering the graph using Leiden algorithm
sc.tl.leiden(adata,resolution=1) 

# Computing UMAP
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['leiden','sample'], title="UMAP Nematostella planula",ncols=1)
sc.pl.umap(adata, color=['leiden'], legend_loc= 'on data')
sc.pl.umap(adata, color=['total_counts', 'n_genes_by_counts'])
sc.pl.umap(adata, color=['log1p_total_counts', 'log1p_n_genes_by_counts'])

In [ ]:
# Checking how each sample contribute to the final map => investigate possible batch effect

sc.pl.umap(adata, color=['leiden'], frameon=False)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Planula 2d'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Planula 3d'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Planula 4d'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Planula 4d cryo'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False,ncols=1)

In [ ]:
# Re-exporting data
adata.write_h5ad(outputPath+'Nematostella_planula.h5ad')

In [ ]:
# Ranking and extracting marker genes for all clusters
sc.tl.rank_genes_groups(adata,'leiden',n_genes=1000,method='wilcoxon')

clusters = np.unique(adata.obs['leiden'])
clusters = clusters.astype('int')
clusters = np.sort(clusters)
clusters = clusters.astype('str')

markers = pd.DataFrame()
clus = []
markerGene = []
padj = []
for i in clusters:
  genes = adata.uns['rank_genes_groups']['names'][str(i)]
  clus += list(np.repeat(i,len(genes)))
  markerGene += list(genes)
  padj += list(adata.uns['rank_genes_groups']['pvals_adj'][str(i)])
markers['clus'] = clus
markers['markerGene'] = markerGene
markers['padj'] = padj
significant = []
for index,row in markers.iterrows():
    if row['padj'] < 0.05:
        significant += "T"
    else:
        significant += "F"
markers['significant'] = significant
significant_markers = markers[markers.significant == "T"]
significant_markers.to_csv(outputPath+'Nematostella_planula_markers_1000g.csv', sep='\t',index=False)
print(significant_markers)

In [ ]:
def cluster_markers(markersDf,clusterNumber):
    subset = markersDf[markersDf["clus"].isin(clusterNumber)]
    markerGene = subset["markerGene"].tolist()
    return markerGene

In [ ]:
# Test
sc.settings.set_figure_params(dpi=90, facecolor='white')
sc.pl.umap(adata,color=['NV2g014334000.1','NV2g024221000.1','NV2g020252000.1','NV2g009892000.1','NV2g020231000.1','NV2g023245000.1','NV2g024302000.1','NV2g003480000.1','NV2g011180000.1','NV2g012780000.1','NV2g025037000.1','NV2g011361000.1','NV2g012332000.1','NV2g015152000.1','NV2g015150000.1','NV2g010021000.1','NV2g009676000.1','NV2g009911000.1'],size=20,ncols=2)

In [ ]:
# Plotting top marker gene for each cluster
markers = []
for i in np.unique(adata.obs['leiden']):
     markers += [adata.uns['rank_genes_groups']['names'][str(i)][0]]
sc.pl.umap(adata,color=markers,color_map = 'viridis')

In [ ]:
# Embryonic ectoderm => TXT51-like 
sc.pl.umap(adata,color=['NV2g004249000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
# Embryonic mesoderm => NvSnailB
sc.pl.umap(adata,color=['NV2g024886000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
# Embryonic endomesoderm
sc.pl.umap(adata,color=['NV2g008037000.1','NV2g023418000.1','NV2g016313000.1','NV2g014212000.1','NV2g016600000.1','NV2g014143000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
# Endomesoderm
sc.pl.umap(adata,color=['NV2g019682000.1','NV2g019627000.1','NV2g003734000.1','NV2g010172000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
# Pharyngeal ectoderm => NvFoxA
sc.pl.umap(adata,color=["NV2g011441000.1","NV2g007354000.1","NV2g022761000.1","NV2g020247000.1","NV2g007661000.1","NV2g019607000.1","NV2g012087000.1","NV2g001654000.1","NV2g025421000.1"],size=20,ncols=2,color_map = 'viridis')

In [ ]:
# Ectoderm epidermis
sc.pl.umap(adata,color=['NV2g013603000.1'],size=20,ncols=2,color_map = 'viridis')

In [ ]:
# Gastroderm => FRIS-like4
sc.pl.umap(adata,color=['NV2g009345000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
# Retractor muscle => Nve-Tpm2 
sc.pl.umap(adata,color=['NV2g019134000.1','NV2g017681000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
# NPC => NvSoxB.2  
sc.pl.umap(adata,color=['NV2g004477000.1','NV2g013863000.1','NV2g017090000.1','NV2g015516000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
# Neuronal => NvAshA
sc.pl.umap(adata,color=['NV2g009665000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
# Secretory => VKT52-like
sc.pl.umap(adata,color=['NV2g008165000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
# Mucous gland 
sc.pl.umap(adata,color=['NV2g012902000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
# Cnidocyte => EVA1C-like3
sc.pl.umap(adata,color=['NV2g005200000.1'],size=20,ncols=2,color_map = 'viridis')

In [ ]:
# Mature cnidocyte => FOS-like
sc.pl.umap(adata,color=['NV2g019749000.1','NV2g016321000.1'],size=20,ncols=2,color_map = 'viridis') 

In [ ]:
sc.tl.paga(adata, groups='leiden')

In [ ]:
adata.obs['leiden_anno'] = adata.obs['leiden']

In [ ]:
adata.obs['leiden_anno'].cat.categories = ['0', '1', '2', '3', '4', '5', '6', '7', '8?', '9', '10', '11', '12','13', '14', '15', '16', '17', '18', '19', '20']

In [ ]:
sc.pl.paga(adata, threshold=0.03, show=False,fontsize=9)

In [ ]:
sc.pl.umap(adata, color=['leiden'], legend_loc= 'on data')

In [ ]:
cluster_markers(significant_markers,['18'])

In [ ]:
adata.obs['leiden_anno'].cat.categories = ["0 - Ectodermis","1 - Ectodermis","2 - Ectodermis","3 - Endomesodermis","4 - Pharyngeal ectodermis","5 - Early cnidocytes","6 - NPC","7 - Ectodermis","8 - Neurons","9 - Cnidocytes","10 - Mucous gland cells","11 - Differentiating cnidocytes","12 - Mature cnidocytes","13 - Neurons","14 - Secretory cells","15 - Ectodermis","16 - Ectodermis","17 - Ectodermis","18 - Neurons","19 - Neurons","20 - Cnidocytes"]

In [ ]:
sc.settings.set_figure_params(dpi=90, facecolor='white')
sc.pl.umap(adata, color=['leiden_anno'],title='Nematostella planula annotated clusters',groups=["5 - Early cnidocytes","9 - Cnidocytes","11 - Differentiating cnidocytes","12 - Mature cnidocytes","20 - Cnidocytes"],ncols=1,frameon=False,save="Nematostella_planula_cnidocytes.pdf")

In [ ]:
sc.settings.set_figure_params(dpi=150, facecolor='white')
sc.pl.umap(adata,title='Nematostella planula\nRBH - one2one - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#A366E0", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#A366E0", "10":"#E2E2E2", "11":"#A366E0", "12":"#A366E0", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2","19":"#E2E2E2","20":"#A366E0"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Nematostella planula\nRBH - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#A366E0", "5":"#A366E0", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#A366E0", "10":"#E2E2E2", "11":"#A366E0", "12":"#A366E0", "13":"#E2E2E2", "14":"#A366E0", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2","19":"#E2E2E2","20":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Nematostella planula\nOrthoFinder - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#A366E0", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#A366E0", "10":"#E2E2E2", "11":"#A366E0", "12":"#A366E0", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2","19":"#E2E2E2","20":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Nematostella planula\nOrthoFinder - many2many - genomeBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#A366E0", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#A366E0", "10":"#E2E2E2", "11":"#A366E0", "12":"#A366E0", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2","19":"#E2E2E2","20":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Nematostella planula\nRBH - one2one - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#0080ff", "9":"#E2E2E2", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#0080ff", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#0080ff","19":"#0080ff","20":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Nematostella planula\nRBH - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#0080ff", "1":"#E2E2E2", "2":"#0080ff", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#0080ff", "9":"#E2E2E2", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#0080ff", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#0080ff","19":"#0080ff","20":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Nematostella planula\nOrthoFinder - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#0080ff", "9":"#E2E2E2", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#0080ff", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#0080ff","19":"#E2E2E2","20":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Nematostella planula\nOrthoFinder - many2many - genomeBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#0080ff", "9":"#E2E2E2", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#0080ff", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#0080ff","19":"#E2E2E2","20":"#E2E2E2"}, frameon=False)